In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-08-21 16:11:02.654 DEBU

In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *

### Configuration parameters

In [3]:
NE_TYPES = ['PERSON']
# NE_TYPES = ['PERSON', 'GPE', 'ORG', 'LOC']
NE_TYPES = ['GPE','DATE','ORG','EVENT','LOC','FAC','CARDINAL','QUANTITY','NORP','ORDINAL','WORK_OF_ART','PERSON']
NE_TYPES = ['EVENT','LOC','FAC','CARDINAL','QUANTITY','NORP','ORDINAL','WORK_OF_ART','PERSON']
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
# MODELS = ['ne_centroid', 'ne_nearest']
MODELS = ['svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}
USE_CV = False

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-21 16:11:14.879 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-08-21 16:11:15.440 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-08-21 16:11:15.681 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-08-21 16:11:15.689 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-08-21 16:11:15.696 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
2018-08-21 16:11:19.488 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 60: [loading embeddings from `/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-21 16:11:19.489 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-21 16:11:19.490 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-21 16:11:48.840 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 266: loaded (400000, 100) matr

In [6]:
from collections import defaultdict
f1_scores = {}
th_list = {}
for ne_type in NE_TYPES:
    print('NE type: {}'.format(ne_type))
    dataset = filter_dataset_by_ne_types(dataset_orig, ne_type, preserveBIO=True)
    f1_scores[ne_type] = {}
    th_list[ne_type] = {'ne_centroid':[], 'ne_nearest':[]}
    for model in MODELS:
        f1_scores[ne_type][model] = {'values':[], 'values_chunk':[]}
#     f1_scores[ne_type]['avg'] = {'values':[]}'ne_centroid', 'ne_nearest'
    for idx_sample in range(N_SAMPLES):
        print('Sample {}'.format(idx_sample+1))
        tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
        tags_train = removeBIOFromTags(tags_train)
        tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
        tags_test_bio = tags_test
        tags_test_bio_flat = flatten_list(tags_test_bio)
        tags_test = removeBIOFromTags(tags_test)
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        ner_bin_clf = FewshotNerBinaryClassifier(embedder)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        if USE_CV:
            cv_res = select_sim_thresholds_with_cv(tokens_train,tags_train, embedder)
        pred_avg = np.zeros(y_test.size)
        n_models_avg = 0
        for model in MODELS:
            if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
                pred= pred_class_labels_bin(flat_sim_one_type(results[model]['sim'], sim_type='cosine'), cv_res[model]['threshold'])
                th_list[ne_type][model].append(cv_res[model]['threshold'])
            else:
                pred = results[model]['pred']
            pred_tags_bio = predToTags(pred, accountBIO=True)
            if model in ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']:   
                pred_avg += pred
                n_models_avg += 1
            f1 = f1_score(y_test, pred)
#             print(tags_test_bio)
#             print(pred_tags_bio)
            f1_chunk = precision_recall_f1_chunks(tags_test_bio_flat, pred_tags_bio, print_results=False)['__total__']['f1']/100
            f1_scores[ne_type][model]['values'].append(f1)
            f1_scores[ne_type][model]['values_chunk'].append(f1_chunk)
        pred_avg /= n_models_avg
#         pred_avg = np.array(map(lambda x: 1 if x >= 0.5 else 0, pred_avg))
#         f1 = f1_score(y_test, pred_avg)
#         f1_scores[ne_type]['avg']['values'].append(f1)
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 scores: {}'.format(cur_f1_scores_dict))
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values_chunk'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 chunk scores: {}'.format(cur_f1_scores_dict))                                          
    for model in MODELS:
        f1_scores[ne_type][model]['mean'] = np.mean(f1_scores[ne_type][model]['values'])
        f1_scores[ne_type][model]['std'] = np.std(f1_scores[ne_type][model]['values'])
        print('{} F1: mean = {} std = {}'.format(model, f1_scores[ne_type][model]['mean'], f1_scores[ne_type][model]['std']))
        f1_scores[ne_type][model]['mean_chunk'] = np.mean(f1_scores[ne_type][model]['values_chunk'])
        f1_scores[ne_type][model]['std_chunk'] = np.std(f1_scores[ne_type][model]['values_chunk'])
        if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
            th_list[ne_type][model] = np.mean(th_list[ne_type][model])

NE type: EVENT
Num of train sentences: 681
Num of valid sentences: 127
Num of test sentences: 56
Sample 1
Train
SVM classifier model
F1 scores: {'svm': 0.7529976019184653}
F1 chunk scores: {'svm': 0.40251572327044016}
Sample 2
Train
SVM classifier model
F1 scores: {'svm': 0.7358916478555305}
F1 chunk scores: {'svm': 0.4418604651162791}
Sample 3
Train
SVM classifier model
F1 scores: {'svm': 0.6960352422907489}
F1 chunk scores: {'svm': 0.3958333333333334}
Sample 4
Train
SVM classifier model
F1 scores: {'svm': 0.6997389033942559}
F1 chunk scores: {'svm': 0.24324324324324323}
Sample 5
Train
SVM classifier model
F1 scores: {'svm': 0.7409470752089136}
F1 chunk scores: {'svm': 0.34013605442176875}
Sample 6
Train
SVM classifier model
F1 scores: {'svm': 0.7371007371007371}
F1 chunk scores: {'svm': 0.36585365853658536}
Sample 7
Train
SVM classifier model
F1 scores: {'svm': 0.7750677506775067}
F1 chunk scores: {'svm': 0.42758620689655175}
Sample 8
Train
SVM classifier model
F1 scores: {'svm': 0.6

SVM classifier model
F1 scores: {'svm': 0.937799043062201}
F1 chunk scores: {'svm': 0.9411764705882352}
Sample 2
Train
SVM classifier model
F1 scores: {'svm': 0.9289099526066351}
F1 chunk scores: {'svm': 0.9371980676328504}
Sample 3
Train
SVM classifier model
F1 scores: {'svm': 0.937799043062201}
F1 chunk scores: {'svm': 0.9371980676328502}
Sample 4
Train
SVM classifier model
F1 scores: {'svm': 0.9519230769230769}
F1 chunk scores: {'svm': 0.9514563106796118}
Sample 5
Train
SVM classifier model
F1 scores: {'svm': 0.9363636363636363}
F1 chunk scores: {'svm': 0.9308755760368663}
Sample 6
Train
SVM classifier model
F1 scores: {'svm': 0.9528301886792453}
F1 chunk scores: {'svm': 0.9519230769230768}
Sample 7
Train
SVM classifier model
F1 scores: {'svm': 0.967741935483871}
F1 chunk scores: {'svm': 0.958139534883721}
Sample 8
Train
SVM classifier model
F1 scores: {'svm': 0.9556650246305419}
F1 chunk scores: {'svm': 0.9651741293532338}
Sample 9
Train
SVM classifier model
F1 scores: {'svm': 0.95

### Print results

In [7]:
print('{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}'.format('MODEL', 'MEAN', 'STD', 'MEAN_CHUNK', 'STD_CHUNK'))
for ne_type in NE_TYPES:
    print('---------------{}---------------'.format(ne_type))
    for model in MODELS:
        print('{:<8s}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}'.format(model, f1_scores[ne_type][model]['mean']*100, f1_scores[ne_type][model]['std']*100,
                                                f1_scores[ne_type][model]['mean_chunk']*100, f1_scores[ne_type][model]['std_chunk']*100))

MODEL   	MEAN    	STD     	MEAN_CHUNK	STD_CHUNK
---------------EVENT---------------
svm     	72.68   	2.85    	36.94   	5.65    
---------------LOC---------------
svm     	65.72   	2.97    	44.92   	4.20    
---------------FAC---------------
svm     	71.93   	1.82    	49.68   	1.67    
---------------CARDINAL---------------
svm     	75.80   	3.71    	64.99   	3.93    
---------------QUANTITY---------------
svm     	78.80   	2.45    	46.62   	7.29    
---------------NORP---------------
svm     	81.90   	3.72    	76.85   	4.04    
---------------ORDINAL---------------
svm     	94.51   	1.23    	94.53   	1.09    
---------------WORK_OF_ART---------------
svm     	69.40   	4.17    	31.21   	9.84    
---------------PERSON---------------
svm     	86.65   	2.73    	73.83   	3.92    


In [8]:
if USE_CV:
    print('Thresholds chosen with cross-validation:')
    print(th_list)